In [ ]:
import osmnx as ox
import networkx as nx
import matplotlib.pyplot as plt


# Define the center of your area of interest and the bounding box size (in meters)
center_point = ox.geocoder.geocode("Stora Torget, Karlstad, Sweden")
dist = 1000  # Distance from the center point in meters to define the bounding box

# Download and project the bike network to SWEREF 99 TM
G = ox.graph_from_point(center_point, dist=dist, network_type='bike', simplify=True, custom_filter='["bicycle"~"yes|designated"]')
G_projected = ox.project_graph(G, to_crs="EPSG:3006")


In [ ]:
fig, ax = ox.plot_graph(G_projected)

### Data overview

Now that we obtained a complete network graph for the travel mode we specified
(cycling), we can take a closer look at which attributes are assigned to the
nodes and edges of the network. It is probably easiest to first convert the
network into a geo-data frame on which we can then use the tools we learnt in
earlier lessons.

To convert a graph into a geo-data frame, we can use `osmnx.graph_to_gdfs()`
(see [previous section](retrieve-data-from-openstreetmap)). Here, we can make
use of the function’s parameters `nodes` and `edges` to select whether we want
only nodes, only edges, or both (the default):

In [ ]:
# Retrieve only edges from the graph
edges = ox.graph_to_gdfs(G_projected, nodes=False, edges=True)
edges.head()

The resulting geo-data frame comprises of a long list of columns. Most of them
relate to [OpenStreetMap tags](https://wiki.openstreetmap.org/wiki/Tags), and
their names are rather self-explanatory. the columns `u` and `v` describe the
topological relationship within the network: they denote the start and end node
of each edge.


* - Column
  - Description
  - Data type
* - [bridge](http://wiki.openstreetmap.org/wiki/Key:bridge)    
  - Bridge feature              
  - boolean           
* - geometry                                                   
  - Geometry of the feature     
  - Shapely.geometry  
* - [highway](http://wiki.openstreetmap.org/wiki/Key:highway)  
  - Tag for roads (road type)   
  - str / list        
* - [lanes](http://wiki.openstreetmap.org/wiki/Key:lanes)      
  - Number of lanes             
  - int (or nan)      
* - [length](http://wiki.openstreetmap.org/wiki/Key:length)    
  - Length of feature (meters)  
  - float             
* - [maxspeed](http://wiki.openstreetmap.org/wiki/Key:maxspeed)
  - maximum legal speed limit   
  - int /list         
* - [name](http://wiki.openstreetmap.org/wiki/Key:name)        
  - Name of the (street) element
  - str (or nan)      
* - [oneway](http://wiki.openstreetmap.org/wiki/Key:oneway)    
  - One way road                
  - boolean           
* - [osmid](http://wiki.openstreetmap.org/wiki/Node)           
  - Unique ids for the element  
  - list              
* - [u](http://ow.ly/bV8n30h7Ufm)                              
  - The start node of edge      
  - int               
* - [v](http://ow.ly/bV8n30h7Ufm)                              
  - The end node of edge       
  - int               
:::


What types of streets does our network comprise of?

In [ ]:
edges["highway"].value_counts()


## Analysing network properties

Now that we have prepared a routable network graph, we can turn to the more
analytical features of OSMnx, and extract information about the network.
To compute basic network characteristics, use
[`osmnx.basic_stats()`](https://osmnx.readthedocs.io/en/stable/osmnx.html#osmnx.stats.basic_stats):

In [ ]:
# Calculate network statistics
ox.basic_stats(G_projected)

This does not yet yield all interesting characteristics of our network, as
OSMnx does not automatically takes the area covered by the network into
consideration. We can do that manually, by, first, delineating the [complex
hull](https://en.wikipedia.org/wiki/Convex_hull) of the network (of an ’unary’
union of all its features), and then, second, computing the area of this hull.

In [ ]:
convex_hull = edges.unary_union.convex_hull
convex_hull

In [ ]:
stats = ox.basic_stats(G_projected, area=convex_hull.area)
stats



## Shortest path analysis

Let’s now calculate the shortest path between two points using
[`osmnx.shortest_path()`](https://osmnx.readthedocs.io/en/stable/osmnx.html?highlight=get_nearest_node#osmnx.distance.shortest_path).



### Origin and destination points 

First we need to specify the source and target locations for our route. If you
are familiar with the area, you can specify a custom placename as a
source location. 

We could figure out the coordinates for these locations manually, and create
`shapely.geometry.Point`s based on the coordinates.  However, if we would have
more than just two points, that would quickly become a chore. Instead, we can
use OSMnx to geocode the locations.

Remember to transform the origin and destination points to the same reference
system as the network data.

In this case let's try to points of interest

In [ ]:

# Geocode the origin and destination to get lat/lon
origin_latlon = ox.geocoder.geocode("Bishops Arm, Karlstad")
destination_latlon = ox.geocoder.geocode("Löfbergs Rosteri & Kaffebar, Karlstad")


In [ ]:
destination_latlon

In [ ]:
origin_latlon

In [ ]:
import geopandas as gpd
from shapely.geometry import Point

# Create GeoDataFrame with the geocoded points
gdf = gpd.GeoDataFrame(
    {'geometry': [Point(origin_latlon[::-1]), Point(destination_latlon[::-1])]}, 
    crs="EPSG:4326"
)

# Transform the GeoDataFrame to SWEREF 99 TM
gdf_transformed = gdf.to_crs("EPSG:3006")

# Extract transformed coordinates
origin_point = gdf_transformed.geometry.iloc[0]
destination_point = gdf_transformed.geometry.iloc[1]

# Use nearest_nodes to find the nearest nodes in the graph to these transformed points
origin_node = ox.distance.nearest_nodes(G_projected, origin_point.x, origin_point.y)
destination_node = ox.distance.nearest_nodes(G_projected, destination_point.x, destination_point.y)


In [ ]:
origin_node

In [ ]:
destination_node

### Routing

Now we are ready for routing and to find the shortest path between the
origin and target locations. We will use
[`osmnx.shortest_path()`](https://osmnx.readthedocs.io/en/stable/osmnx.html?highlight=get_nearest_node#osmnx.distance.shortest_path).

The function accepts three mandatory parameters: a graph, an origin node id, and
a destination node id, and two optional parameters: `weight` can be set to
consider a different *cost impedance* than the length of the route, and `cpus`
controls parallel computation of many routes.

In [ ]:
# Find the shortest path by length
shortest_path = nx.shortest_path(G_projected, origin_node, destination_node, weight='length')



In [ ]:
# Plot the bike network and the shortest path
fig, ax = ox.plot_graph_route(G_projected, shortest_path, route_color='blue', route_linewidth=6, node_size=0, bgcolor='k')


In [ ]:
# Find the shortest path by length
shortest_path = nx.shortest_path(G_projected, origin_node,destination_node)
# Plot the bike network and the shortest path
fig, ax = ox.plot_graph_route(G_projected, shortest_path, route_color='blue', route_linewidth=6, node_size=0, bgcolor='k')


In [ ]:
shortest_path

As a result we get a list of all the nodes that are along the shortest path. 

We could extract the locations of those nodes from the `nodes`
GeoDataFrame and create a LineString presentation of the points, but luckily,
OSMnx can do that for us and we can plot shortest path by using
`plot_graph_route()` function:

In [ ]:
# Plot the shortest path
fig, ax = ox.plot_graph_route(G_projected, shortest_path)

In [ ]:
# Extract lengths of each segment in the shortest path
lengths = ox.utils_graph.get_route_edge_attributes(G_projected, shortest_path, "length")

# Calculate the total length in meters
total_length = sum(lengths)
print(f"Total length of the shortest path: {total_length:.2f} meters")



## Saving shortest paths to disk

Quite often you need to save the route into a file for further analysis and
visualization purposes, or at least have it as a GeoDataFrame object in Python.
Hence, let's continue still a bit and see how we can turn the route into a
linestring and save the shortest path geometry and related attributes into a
geopackage file.

First we need to get the nodes that belong to the shortest path:

In [ ]:
# Get the nodes along the shortest path
nodes_gdf = ox.graph_to_gdfs(G_projected, edges=False)

# Locate the route nodes in the nodes GeoDataFrame
route_nodes = nodes_gdf.loc[shortest_path]

In [ ]:
route_nodes

As we can see, now we have all the nodes that were part of the shortest path as a GeoDataFrame.

Now we can create a LineString out of the Point geometries of the nodes:

In [ ]:
import shapely.geometry

# Create a geometry for the shortest path
route_line = shapely.geometry.LineString(
    list(route_nodes.geometry.values)
)
route_line

In [ ]:
import geopandas

route_geom = geopandas.GeoDataFrame(
    {
        "geometry": [route_line],
        "osm_nodes": [shortest_path],
    },
    crs=edges.crs
)

# Calculate the route length
route_geom["length_m"] = route_geom.length

route_geom.head()

In [ ]:
# Geocode the location to get its latitude and longitude
location_point = ox.geocoder.geocode("Stora Torget, Karlstad, Sweden")

# Define the distance by which to extend the bounding box (in meters)
distance = 1000  # Extend by 1000 meters

# Calculate the bounding box around the location, extended by the specified distance
north, south, east, west = ox.utils_geo.bbox_from_point(
    point=location_point, dist=distance)

# You can now use this bounding box to get the graph
graph = ox.graph_from_bbox(north, south, east, west, network_type='all')



print(f"Bounding Box: North: {north}, South: {south}, East: {east}, West: {west}")


In [ ]:

# Download building geometries within the specified bounding box
buildings = ox.geometries.geometries_from_bbox(north, south, east, west, tags={"building": True})



In [ ]:
buildings.plot()

In [ ]:
import contextily as ctx
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(12, 8))

# Reproject data to Web Mercator (EPSG:3857) for consistency with contextily basemap
edges_3857 = edges.to_crs(epsg=3857)
nodes_3857 = nodes_gdf.to_crs(epsg=3857)
buildings_3857 = buildings.to_crs(epsg=3857)
route_geom_3857 = route_geom.to_crs(epsg=3857)

# Plot edges and nodes
edges_3857.plot(ax=ax, linewidth=0.75, color='gray')
nodes_3857.plot(ax=ax, markersize=2, color='gray')

# Add buildings
buildings_3857.plot(ax=ax, facecolor='yellow', alpha=0.7)

# Add the route
route_geom_3857.plot(ax=ax, linewidth=2, linestyle='--', color='red')

# Add basemap
ctx.add_basemap(ax, source=ctx.providers.CartoDB.Positron)

# Adjust the plot limits to the extent of your data
ax.set_xlim(edges_3857.total_bounds[[0, 2]].min(), edges_3857.total_bounds[[0, 2]].max())
ax.set_ylim(edges_3857.total_bounds[[1, 3]].min(), edges_3857.total_bounds[[1, 3]].max())

plt.show()


## Sources

This lesson is inspired and has adapted or reused material from University of Helsinki Automating GIS processis course (https://autogis-site.readthedocs.io/en/latest/course-info/license.html) under a Creative Commons Attribution-ShareAlike 4.0 International licence (https://creativecommons.org/licenses/by-sa/4.0/deed.en).